# Identifying Gray Matter Markers of Irritability: a machine learning approach
This notebook is designed to analyze previously processed gray matter density volumes using support vector regression.

In [2]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm.preprocess import VBMSegment, Segment
from nipype.interfaces.ants import Atropos, Registration, ApplyTransforms, N4BiasFieldCorrection
from nipype.interfaces.fsl import ApplyMask, BET
from pandas import DataFrame, Series, read_csv

# Study specific variables
study_home = '/moochie/Cat/Aggregate_anats/GMD_ML'

sub_data_file = study_home + '/doc/subject_info.csv'
subject_info = read_csv(sub_data_file)
subjects_list = subject_info['freesurferID'].tolist()

preproc_dir = study_home + '/proc'
output_dir = study_home + '/ml_trainingset'

sample_template = study_home + '/templates/lcbd_template_1mm.nii.gz'
sample_template_brain = study_home + '/templates/lcbd_template_1mm_brain.nii.gz'
sample_template_mask = study_home + '/templates/lcbd_template_1mm_mask.nii.gz'


subject_info['age_inv'] = 1/subject_info['Age_yrs']
subject_info.describe()

,SubjectID,Sequence Version,Potentially Usable,in_analysis,Age_yrs,Age_mos,male,MAP_Temper_Loss,MAP_Noncompliance,MAP_General_Aggression,MAP_Low_Concern,age_inv
count,170.000000,170.000000,170.0,170.0,170.000000,170.000000,170.000000,158.000000,158.000000,158.000000,158.000000,170.000000
mean,833.394118,2.335294,1.0,1.0,7.780763,92.841176,0.535294,18.373418,3.971160,2.942532,3.118847,0.136446
std,727.043183,1.211113,0.0,0.0,1.903593,22.834977,0.500226,17.809138,3.690790,3.244171,3.342947,0.034174
min,2.000000,1.000000,1.0,1.0,4.062971,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080222
25%,128.250000,1.000000,1.0,1.0,6.483915,77.000000,0.000000,5.000000,1.142857,0.520000,0.444444,0.110889
50%,1005.500000,2.000000,1.0,1.0,7.578371,90.500000,1.000000,13.000000,2.882727,1.960000,2.000000,0.131955
75%,1032.750000,4.000000,1.0,1.0,9.018480,107.500000,1.000000,28.000000,6.136364,4.000000,4.444444,0.154228
max,2038.000000,4.000000,1.0,1.0,12.465435,149.000000,1.000000,81.000000,15.429091,13.600000,15.111111,0.246125


In [5]:
from sklearn.preprocessing import StandardScaler, PowerTransformer
from numpy import squeeze

## Create a conditions list for the feature set
age_labels = subject_info[['Age_yrs','age_inv']].copy()
age_labels = age_labels.values
irr_labels = subject_info[['MAP_Temper_Loss','MAP_Noncompliance','MAP_General_Aggression','MAP_Low_Concern']].copy()
irr_labels = irr_labels.values

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(age_labels)
sd_agedata = scaler.transform(age_labels)

pt = PowerTransformer()
pt.fit(irr_labels)
pt_irritability = pt.transform(irr_labels)
pt_irritability = squeeze(pt_irritability)

conditions = DataFrame(data=sd_agedata, index=None, columns=['age','ageinv'])
conditions['subject'] = Series(subjects_list, index=conditions.index)
conditions = conditions.merge(DataFrame(pt_irritability,columns=['Temper_Loss','Noncompliance','General_Aggression','Low_Concern']))

#conditions.to_csv(output_dir + '/featureset_key.csv')
conditions.describe()

ValueError: can not merge DataFrame with instance of type <class 'numpy.ndarray'>

In [6]:
## Concatenate all the parameter estimates from preproc to create a feature set
from glob import glob
from nipype.interfaces.fsl.utils import Merge
from nipype.interfaces.fsl import SUSAN
files = glob(preproc_dir + '/soft_tissue_files/*/POSTERIOR_02.nii.gz')
files = sorted(files)

def brightthresh(img):
    import nibabel as nib
    from numpy import median, where
    
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    img_nifti1 = nib.load(img)
    img_data = img_nifti1.get_data()
    img_data = img_data.astype(float)
    
    brain_values = where(img_data > 0)
    median_thresh = median(brain_values)
    bright_thresh = 0.75 * median_thresh
    
    return(bright_thresh)

sm = SUSAN()
sm.inputs.fwhm=6

for file in files:
    sm.inputs.brightness_threshold = brightthresh(file)
    sm.inputs.in_file = file
    sm.inputs.out_file = file.replace('POSTERIOR_02','smoothed_gm')
    sm.run()

gm_files = glob(preproc_dir + '/soft_tissue_files/*/smoothed_gm.nii.gz')
gm_files = sorted(gm_files)
gmd_feature_data = output_dir + '/gmd_smooth_combined.nii.gz'

merge = Merge()
merge.inputs.in_files = gm_files
merge.inputs.dimension = 't'
merge.inputs.merged_file = gmd_feature_data
#merge.run()

181120-13:35:10,139 interface DEBUG:
	 brightness_threshold_98.25
181120-13:35:10,143 interface DEBUG:
	 dimension_3
181120-13:35:10,144 interface DEBUG:
	 in_file_/moochie/Cat/Aggregate_anats/GMD_ML/proc/soft_tissue_files/101/POSTERIOR_02.nii.gz
181120-13:35:10,145 interface DEBUG:
	 out_file_/moochie/Cat/Aggregate_anats/GMD_ML/proc/soft_tissue_files/101/smoothed_gm.nii.gz
181120-13:35:10,146 interface DEBUG:
	 use_median_1
181120-13:38:26,616 interface DEBUG:
	 brightness_threshold_98.25
181120-13:38:26,619 interface DEBUG:
	 dimension_3
181120-13:38:26,620 interface DEBUG:
	 in_file_/moochie/Cat/Aggregate_anats/GMD_ML/proc/soft_tissue_files/102/POSTERIOR_02.nii.gz
181120-13:38:26,621 interface DEBUG:
	 out_file_/moochie/Cat/Aggregate_anats/GMD_ML/proc/soft_tissue_files/102/smoothed_gm.nii.gz
181120-13:38:26,622 interface DEBUG:
	 use_median_1
181120-13:41:42,981 interface DEBUG:
	 brightness_threshold_98.25
181120-13:41:42,983 interface DEBUG:
	 dimension_3
181120-13:41:42,984 inter

In [7]:
analysis = 'irritability'

if analysis == 'age':
    labels = conditions['age']
    type_svm = 'regression'
elif analysis == 'ageinv':
    labels = conditions['ageinv']
    type_svm = 'regression'
elif analysis == 'irritability':
    labels = conditions['irritability']
    type_svm = 'regression'

results_file = open(output_dir + '/results_' + analysis + '.txt','w')
labels.describe()

count    1.170000e+02
mean    -5.427757e-16
std      1.004301e+00
min     -1.867147e+00
25%     -7.193634e-01
50%     -4.972326e-02
75%      6.939497e-01
max      2.052883e+00
Name: irritability, dtype: float64

In [8]:
if type_svm == 'regression':
    # Perform the support vector classification
    from nilearn.input_data import NiftiMasker
    from sklearn.feature_selection import f_regression, SelectPercentile
    from sklearn.svm import SVR
    from sklearn.pipeline import Pipeline

    # Set up the regression
    svr = SVR(kernel='linear', C=1)
    masker = NiftiMasker(mask_img=sample_template_mask,standardize=True, 
                         memory='nilearn_cache', memory_level=1)
    
    feature_selection = SelectPercentile(f_regression, percentile=5)
    fs_svr = Pipeline([('feat_select', feature_selection), ('svr', svr)])
    
    # Run the regression
    X = masker.fit_transform(gmd_feature_data)
    fs_svr.fit(X, labels)
        
    from sklearn.model_selection import cross_val_predict, LeaveOneGroupOut

    loso = LeaveOneGroupOut()
    y_pred = cross_val_predict(fs_svr, X, y=labels, n_jobs=6,
                               groups=conditions['subject'],cv=loso)
    # save weights
    coef = svr.coef_
    coef = feature_selection.inverse_transform(coef)
    coef_image = masker.inverse_transform(coef)
    coef_image.to_filename(output_dir + '/svrweights_' + analysis + '.nii.gz')
    
    from scipy.stats import linregress
    slope, intercept, r_val, p_val, stderr = linregress(labels, y_pred) 

    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(labels, y_pred)
    
    from scipy.stats import spearmanr
    spear_r, spear_p = spearmanr(labels, y_pred)

    print("prediction accuracy: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f" % (r_val, p_val, mse, spear_r, spear_p))

    # plot the predicted versus actual values
    import matplotlib.pyplot as plt
    plt.scatter(labels, y_pred, color='b')
    plt.xlabel('Actual ' + analysis)
    plt.ylabel('Predicted ' + analysis)
    plt.savefig(output_dir + '/scatter_pred_actual_' + analysis + '_final.svg', transparent=True)
    plt.show()
    plt.close()

    results_file.write("Prediction accuracy r-value: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f \n" % (r_val, p_val, mse, spear_r, spear_p))
    results_file.write('predicted: ' + str(y_pred) + '\n')
    results_file.write('actual: ' + str(labels) + '\n')

    results_file.close()

prediction accuracy: 0.2301 / p-value: 0.012557 / MSE: 1.291068 // Spearman: 0.184950 / p-value: 0.045900


<Figure size 640x480 with 1 Axes>